# Creating Distance Matrices

In [2]:
## Modules
import geopandas as gpd
import pandas as pd
import numpy as np

import json
import urllib.request
import geopandas as gpd
import pandas as pd
import numpy as np
#import geopy.distance

In [64]:

#cars_by_county = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header=0)
#cars_by_county

## All Locations (FQHCs and PPs)

### Defining Functions

In [ ]:
health_centers = pd.read_csv(r'facilities_w_county.csv', index_col = 0, header=0)
health_centers.index.tolist()
county_pop = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header = 0)

In [3]:
## Utils
'''health_centers = pd.read_csv(r'facilities_w_county.csv', index_col = 0, header=0)
health_centers.index.tolist()
county_pop = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header = 0)'''

def get_county(site, health_centers):
    return health_centers.loc[site]['county']
def get_pop(county_name):
    # Remove "County" from the end and convert to lowercase for matching
    county_normalized = county_name.lower().replace(" county", "").strip()
    
    # Find matching county in county_pop dataframe (case-insensitive)
    county_match = county_pop[county_pop['Counties'].str.lower() == county_normalized]
    
    # Check if match was found
    if county_match.empty:
        print(f"No match found for county: {county_name} (normalized: {county_normalized})")
        return 0  # Return 0 instead of None
    
    # Return the population value
    return int(county_match['Total_Population'].iloc[0])
def get_car_prop(site, health_centers):
    county = get_county(site, health_centers)
    # Remove "County" from the end and convert to lowercase for matching
    county_normalized = county.lower().replace(" county", "").strip()
    
    # Find the car proportion for this county in county_pop
    county_match = county_pop[county_pop['Counties'].str.lower() == county_normalized]
    
    if county_match.empty:
        print(f"No match found for county: {county} (normalized: {county_normalized})")
        return 0.0  # Return a default value instead of None
    
    return county_match['Vehicle_Registration_Proportion'].iloc[0]


'''def get_county(site, health_centers): #changed from get_zip to get_county
    #return health_centers.loc[site]['county']
    county = get_county(site, health_centers)
    # Find the car proportion for this county in county_pop
    county_match = county_pop[county_pop['Counties'].str.lower() == county.lower()]
    if county_match.empty:
        print(f"No match found for county: {county}")
        return None
    return county_match['Vehicle_Registration_Proportion'].iloc[0]'''

def calc_dtilde_matrix(health_centers, t_car, t_pub, t_walk):
    N = health_centers.shape[0]
    dtilde_matrix = np.zeros((N,N))
    
    index_to_pos = {idx: pos for pos, idx in enumerate(health_centers.index)}
    
    for i, a in health_centers.iterrows():
        for j, b in health_centers.iterrows():
            if i != j:
                # for walking 
                pos_i = index_to_pos[i]
                pos_j = index_to_pos[j]
                
                if (t_walk.iloc[pos_i,pos_j] == 0 and t_walk.iloc[pos_j,pos_i] == 0):
                    walk = 0
                elif t_walk.iloc[pos_i,pos_j]==0:
                    walk=2*t_walk[pos_j,i]
                elif t_walk.iloc[pos_j,pos_i] == 0:
                    walk=2*t_walk.iloc[pos_i, pos_j]
                else: 
                    walk = t_walk.iloc[pos_i,pos_j] + t_walk.iloc[pos_j,pos_i] 
                # for public transport
                if t_pub.iloc[pos_i,pos_j] == 0 and t_pub.iloc[pos_j,pos_i] == 0:
                    pub = 0
                elif t_pub.iloc[pos_i,pos_j] == 0:
                    pub = 2*t_pub.iloc[pos_j,pos_i]
                elif t_pub.iloc[pos_j,pos_i] == 0:
                    pub = 2*t_pub.iloc[pos_i,pos_j]
                else:
                    pub = t_pub.iloc[pos_i,pos_j] + t_pub.iloc[pos_j,pos_i]
                # for car
                if t_car.iloc[pos_i,pos_j] == 0 and t_car.iloc[pos_j,pos_i] == 0:
                    car = 0
                elif t_car.iloc[pos_i,pos_j] == 0:
                    car = 2*t_car.iloc[pos_j,pos_i]
                elif t_car.iloc[pos_j,pos_i] == 0:
                    car=2*t_car.iloc[pos_i,pos_j]
                else:
                    car = t_car.iloc[pos_i,pos_j] + t_car.iloc[pos_j,pos_i]
                    
                driver_min = min(walk, pub, car)
                non_driver_min = min(walk, pub)
                car_prop = get_car_prop(i, health_centers)
                
                dtilde_matrix[pos_i,pos_j] = car_prop*driver_min + (1-car_prop)*non_driver_min
    return dtilde_matrix

def calc_d_matrix(dtilde_matrix, health_centers):
    N = health_centers.shape[0]
    d_matrix = np.zeros((N,N))
    index_to_pos = {idx: pos for pos, idx in enumerate(health_centers.index)}
    
    for i, a in health_centers.iterrows():
        for j, b in health_centers.iterrows():
            if i != j:
                pos_i = index_to_pos[i]
                pos_j = index_to_pos[j]
                county_i = get_county(i, health_centers)
                county_j = get_county(j, health_centers)
                
                #popi = get_pop(county_i)
                #popj = get_pop(county_j)
                popi = get_pop(county_i) # ERROR
                popj = get_pop(county_j)
                if popi == 0 and popj == 0:
                    d_matrix[pos_i,pos_j] = 0
                elif popi == 0:
                    d_matrix[pos_i,pos_j] = dtilde_matrix[pos_j,pos_i]
                elif popj == 0:
                    d_matrix[pos_i,pos_j] = dtilde_matrix[pos_i,pos_j]
                else:
                    d_matrix[pos_i,pos_j] = (1/(popi+popj))*(popi*dtilde_matrix[pos_i,pos_j] + popj*dtilde_matrix[pos_j,pos_i])

    return d_matrix

In [13]:
health_centers = pd.read_csv('facilities_w_county.csv',
                        index_col=0,
                        header=0)

### Distance Matrix for all 411 locations

In [ ]:
t_car = pd.read_csv("drive_matrix_all.csv",
                        index_col=0,
                        header=0)
t_pub = pd.read_csv("transit_matrix_all.csv",
                        index_col=0,
                        header=0)
# slc_twalk = calc_twalk_matrix(slc_sites, slc_neighbours)
t_walk = pd.read_csv("walking_matrix_all.csv",
                        index_col=0,
                        header=0)

In [54]:
all_loc_dtilde = calc_dtilde_matrix(health_centers, t_car, t_pub, t_walk)
all_loc_d_matrix = calc_d_matrix(all_loc_dtilde, health_centers)

### Loading matrix and saving to CSV for All Locations

In [1]:
#np.save("all_d_matrix.npy", all_loc_d_matrix) 

NameError: name 'np' is not defined

In [ ]:
data = np.load("all_loc_d_matrix.npy")
df = pd.DataFrame(data)
print(df.head())
df = df.fillna(0)
print(df.head())
df.to_csv('dtilde_matrix_all.csv', index=False)

   0    1           2    3           4           5    6    7    8    9    ...  \
0  0.0  NaN         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
1  NaN  0.0         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
2  NaN  NaN    0.000000  NaN  903.671334         NaN  NaN  NaN  NaN  NaN  ...   
3  NaN  NaN         NaN  0.0         NaN         NaN  NaN  NaN  NaN  NaN  ...   
4  NaN  NaN  903.671334  NaN    0.000000  245.742626  NaN  NaN  NaN  NaN  ...   

   401         402        403  404  405  406  407  408  409         410  
0  NaN         NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN  696.437903  
1  NaN         NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
2  NaN  232.914218        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
3  NaN         NaN  88.047353  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
4  NaN  150.470140        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  

[5 rows x 411 columns]
   0    1           2    3           4       

### Distance Matrix for FQHCs only

In [22]:
health_centers = pd.read_csv(r'facilities_w_county.csv', index_col = 0, header=0, nrows=301)
print(health_centers.tail())
health_centers.index.tolist()
county_pop = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header = 0)

                                              address          census tract  \
FQHC 297   9436 E Slauson Ave,Pico Rivera,Los Angeles  Census Tract 5026.03   
FQHC 298               950 Circle Dr,Salinas,Monterey     Census Tract 5.02   
FQHC 299         950 S Euclid Ave,San Diego,San Diego    Census Tract 33.05   
FQHC 300          9933 MacArthur Blvd,Oakland,Alameda     Census Tract 4102   
FQHC 301  995 Potrero Ave,San Francisco,San Francisco   Census Tract 228.02   

                        county                         coordinates  
FQHC 297    Los Angeles County  -118.096877042234, 33.968377302046  
FQHC 298       Monterey County  -121.626232969408, 36.679436931946  
FQHC 299      San Diego County  -117.085175971449, 32.696326773932  
FQHC 300        Alameda County  -122.156742443935, 37.746482609226  
FQHC 301  San Francisco County   -122.40663821554, 37.756926145551  


In [ ]:
# changing xlsx to csv 
df = pd.read_excel("drive_minutes_matrix_fqhc.xlsx", index_col=0, header=0, nrows = 301, )
df.to_csv("drive_minutes_matrix_fqhc.csv",index = True)
data = pd.read_csv("drive_minutes_matrix_fqhc.csv", index_col=0, header=0, nrows=301)
data = data.fillna(0)

In [ ]:
print(data.head())
data.to_csv("drive_matrix_fqhc.csv")


       FQHC1  FQHC2      FQHC3  FQHC4      FQHC5      FQHC6  FQHC7  FQHC8  \
FQHC1    0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0    0.0   
FQHC2    0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0    0.0   
FQHC3    0.0    0.0   0.000000    0.0  72.633333  75.533333    0.0    0.0   
FQHC4    0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0    0.0   
FQHC5    0.0    0.0  73.583333    0.0   0.000000  62.116667    0.0    0.0   

       FQHC9  FQHC10  ...  FQHC293  FQHC294    FQHC295    FQHC296  FQHC297  \
FQHC1    0.0     0.0  ...      0.0      0.0  52.766667   0.000000      0.0   
FQHC2    0.0     0.0  ...      0.0      0.0   0.000000   0.000000      0.0   
FQHC3    0.0     0.0  ...      0.0      0.0   0.000000  84.533333      0.0   
FQHC4    0.0     0.0  ...      0.0      0.0   0.000000   0.000000      0.0   
FQHC5    0.0     0.0  ...      0.0      0.0   0.000000  31.183333      0.0   

       FQHC298    FQHC299  FQHC300  FQHC301  \
FQHC1      0.0   0.00

In [31]:
data = pd.read_csv("drive_matrix_fqhc.csv")
print(data.head())

       Unnamed: 0  FQHC1      FQHC2  FQHC3      FQHC4      FQHC5  FQHC6  \
FQHC1         0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0   
FQHC2         0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0   
FQHC3         0.0    0.0   0.000000    0.0  72.633333  75.533333    0.0   
FQHC4         0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0   
FQHC5         0.0    0.0  73.583333    0.0   0.000000  62.116667    0.0   

       FQHC7  FQHC8  FQHC9  ...  FQHC292  FQHC293    FQHC294    FQHC295  \
FQHC1    0.0    0.0    0.0  ...      0.0      0.0  52.766667   0.000000   
FQHC2    0.0    0.0    0.0  ...      0.0      0.0   0.000000   0.000000   
FQHC3    0.0    0.0    0.0  ...      0.0      0.0   0.000000  84.533333   
FQHC4    0.0    0.0    0.0  ...      0.0      0.0   0.000000   0.000000   
FQHC5    0.0    0.0    0.0  ...      0.0      0.0   0.000000  31.183333   

       FQHC296  FQHC297    FQHC298  FQHC299  FQHC300  FQHC301  
FQHC1      0.0      0.0   0.000000

In [32]:
fqhc_t_car = pd.read_csv("drive_matrix_fqhc.csv",
                        index_col=0,
                        header=0)
fqhc_t_pub = pd.read_csv("transit_matrix_fqhc.csv",
                        index_col=0,
                        header=0)
fqhc_t_walk = pd.read_csv("walking_matrix_fqhc.csv",
                        index_col=0,
                        header=0)

In [33]:
fqhc_loc_dtilde = calc_dtilde_matrix(health_centers, fqhc_t_car, fqhc_t_pub, fqhc_t_walk)
fqhc_loc_d_matrix = calc_d_matrix(fqhc_loc_dtilde, health_centers)

In [34]:
np.save("fqhc_d_matrix.npy", fqhc_loc_d_matrix) 
data = np.load("fqhc_d_matrix.npy")
df = pd.DataFrame(data)
print(df.head())
df = df.fillna(0)
print(df.head())
df.to_csv('dtilde_matrix_fqhc.csv', index=False)

   0    1           2    3           4           5    6    7    8    9    ...  \
0  0.0  NaN         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
1  NaN  0.0         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
2  NaN  NaN    0.000000  NaN  448.153359  352.401819  NaN  NaN  NaN  NaN  ...   
3  NaN  NaN         NaN  0.0         NaN         NaN  NaN  NaN  NaN  NaN  ...   
4  NaN  NaN  448.153359  NaN    0.000000  231.225947  NaN  NaN  NaN  NaN  ...   

   291  292  293         294          295  296  297         298  299  300  
0  NaN  NaN  NaN  207.710656          NaN  NaN  NaN         NaN  NaN  NaN  
1  NaN  NaN  NaN         NaN          NaN  NaN  NaN         NaN  NaN  NaN  
2  NaN  NaN  NaN         NaN  1612.595538  NaN  NaN         NaN  NaN  NaN  
3  NaN  NaN  NaN         NaN          NaN  NaN  NaN  133.307777  NaN  NaN  
4  NaN  NaN  NaN         NaN   494.248162  NaN  NaN         NaN  NaN  NaN  

[5 rows x 301 columns]
   0    1           2    3       